In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
import torch.nn as nn

In [3]:
class Model(nn.Module): ## Inherit from the base class nn.Module

    def __init__(self, num_features):
        super().__init__()
        self.linear = nn.Linear(num_features, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, features):
        out = self.linear(features)
        out = self.sigmoid(out)
        return out

In [4]:
## generating random dataset
features = torch.rand(10, 5)

model = Model(features.shape[1])

model(features) ## NOT NEEDED TO DO model.forward(features)

tensor([[0.4583],
        [0.4120],
        [0.3610],
        [0.4406],
        [0.4466],
        [0.4790],
        [0.4898],
        [0.4706],
        [0.3748],
        [0.4752]], grad_fn=<SigmoidBackward0>)

In [5]:
model.linear.weight

Parameter containing:
tensor([[-0.1376,  0.2986, -0.1022,  0.3447, -0.4360]], requires_grad=True)

In [6]:
model.linear.bias

Parameter containing:
tensor([-0.2973], requires_grad=True)

In [7]:
from torchinfo import summary

In [8]:
summary(model, input_size=(10, 5))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [10, 1]                   --
├─Linear: 1-1                            [10, 1]                   6
├─Sigmoid: 1-2                           [10, 1]                   --
Total params: 6
Trainable params: 6
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

In [10]:
class ANN(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.linear1 = nn.Linear(num_features, 3) ## 3 neurons in hidden layer
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(3, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, features):
        out = self.linear1(features)
        out = self.relu(out)
        out = self.linear2(out)
        out = self.sigmoid(out)
        return out

In [11]:
features = torch.rand(10, 5)

model = ANN(features.shape[1])

In [12]:
model(features)

tensor([[0.5580],
        [0.5664],
        [0.5580],
        [0.5603],
        [0.5505],
        [0.5580],
        [0.6171],
        [0.5796],
        [0.5845],
        [0.5679]], grad_fn=<SigmoidBackward0>)

In [13]:
model.linear1.weight

Parameter containing:
tensor([[-0.2449,  0.2946, -0.3984, -0.3389,  0.3004],
        [ 0.3590, -0.2259,  0.3145,  0.1619, -0.4302],
        [ 0.4224, -0.2112,  0.2726, -0.2231, -0.1493]], requires_grad=True)

In [14]:
summary(model)

Layer (type:depth-idx)                   Param #
ANN                                      --
├─Linear: 1-1                            18
├─ReLU: 1-2                              --
├─Linear: 1-3                            4
├─Sigmoid: 1-4                           --
Total params: 22
Trainable params: 22
Non-trainable params: 0

In [15]:
class ANN2(nn.Module):
    ## using Sequential Container
    def __init__(self, num_features):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(num_features, 3),
            nn.ReLU(),
            nn.Linear(3, 1),
            nn.Sigmoid()
        )

    def forward(self, features):
        out = self.network(features)
        return out

In [16]:
features = torch.rand(10, 5)

model = ANN2(features.shape[1])

In [17]:
model(features)

tensor([[0.3697],
        [0.3536],
        [0.3753],
        [0.3678],
        [0.4038],
        [0.3553],
        [0.3762],
        [0.3536],
        [0.3761],
        [0.3378]], grad_fn=<SigmoidBackward0>)

In [18]:
summary(model)

Layer (type:depth-idx)                   Param #
ANN2                                     --
├─Sequential: 1-1                        --
│    └─Linear: 2-1                       18
│    └─ReLU: 2-2                         --
│    └─Linear: 2-3                       4
│    └─Sigmoid: 2-4                      --
Total params: 22
Trainable params: 22
Non-trainable params: 0

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [20]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')

In [21]:
df.drop(columns= ['id', 'Unnamed: 32'], inplace=True)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

In [25]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [26]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [27]:
X_train_tensor = torch.from_numpy(X_train)
y_train_tensor = torch.from_numpy(y_train)
X_test_tensor = torch.from_numpy(X_test)
y_test_tensor = torch.from_numpy(y_test)

In [28]:
class MySimpleNN(nn.Module):

    def __init__(self, num_features):
        super().__init__()
        self.linear = nn.Linear(num_features, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, features):
        out = self.linear(features)
        out = self.sigmoid(out)
        return out

    def loss_function(self, y_pred, y):
        ## Clamp predictions to avoid log(0)
        epsilon = 1e-7
        y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)

        ## Calculate loss
        loss = -(y * torch.log(y_pred) + (1 - y) * torch.log(1 - y_pred)).mean()
        return loss

In [29]:
learning_rate = 0.1
epochs = 25

In [37]:
model = MySimpleNN(X_train_tensor.shape[1])

In [38]:
for epoch in range(epochs):
    y_pred = model(X_train_tensor.to(torch.float32))
    loss = model.loss_function(y_pred, y_train_tensor)

    loss.backward()

    with torch.no_grad():
        model.linear.weight -= learning_rate * model.linear.weight.grad
        model.linear.bias -= learning_rate * model.linear.bias.grad
    
    model.linear.weight.grad.zero_()
    model.linear.bias.grad.zero_()

    print(f'Epoch: {epoch + 1}, Loss: {loss}')

Epoch: 1, Loss: 0.7548062801361084
Epoch: 2, Loss: 0.7481001019477844
Epoch: 3, Loss: 0.7420361638069153
Epoch: 4, Loss: 0.7365206480026245
Epoch: 5, Loss: 0.7314780354499817
Epoch: 6, Loss: 0.7268463969230652
Epoch: 7, Loss: 0.7225750088691711
Epoch: 8, Loss: 0.718622088432312
Epoch: 9, Loss: 0.7149527072906494
Epoch: 10, Loss: 0.7115377187728882
Epoch: 11, Loss: 0.7083523273468018
Epoch: 12, Loss: 0.7053754925727844
Epoch: 13, Loss: 0.7025890350341797
Epoch: 14, Loss: 0.699977457523346
Epoch: 15, Loss: 0.6975268125534058
Epoch: 16, Loss: 0.6952248215675354
Epoch: 17, Loss: 0.6930607557296753
Epoch: 18, Loss: 0.6910250186920166
Epoch: 19, Loss: 0.6891084909439087
Epoch: 20, Loss: 0.6873031854629517
Epoch: 21, Loss: 0.6856018900871277
Epoch: 22, Loss: 0.6839979887008667
Epoch: 23, Loss: 0.6824849843978882
Epoch: 24, Loss: 0.6810572147369385
Epoch: 25, Loss: 0.6797094941139221


In [41]:
with torch.no_grad():
    y_pred = model.forward(X_test_tensor.to(torch.float32))

In [42]:
y_pred = (y_pred > 0.5).float()

In [43]:
accuracy = (y_pred == y_test_tensor).float().mean()

In [44]:
accuracy

tensor(0.5499)

In [45]:
loss_function = nn.BCELoss()

In [46]:
class MySimpleNN(nn.Module):
    ## Using built-in loss function
    def __init__(self, num_features):
        super().__init__()
        self.linear = nn.Linear(num_features, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, features):
        out = self.linear(features)
        out = self.sigmoid(out)
        return out

In [48]:
y_train_tensor.shape

torch.Size([455])

In [51]:
y_pred.shape

torch.Size([455, 1])

In [53]:
y_pred.dtype

torch.float32

In [55]:
y_train_tensor.dtype

torch.int64

In [56]:
model = MySimpleNN(X_train.shape[1])
for epoch in range(epochs):
    y_pred = model(X_train_tensor.to(torch.float32))
    loss = loss_function(y_pred, (y_train_tensor.to(torch.float32)).view(-1, 1))

    loss.backward()

    with torch.no_grad():
        model.linear.weight -= learning_rate * model.linear.weight.grad
        model.linear.bias -= learning_rate * model.linear.bias.grad
    
    model.linear.weight.grad.zero_()
    model.linear.bias.grad.zero_()

    print(f'Epoch: {epoch + 1}, Loss: {loss}')

Epoch: 1, Loss: 0.6233302354812622
Epoch: 2, Loss: 0.48887813091278076
Epoch: 3, Loss: 0.4146709740161896
Epoch: 4, Loss: 0.3670973777770996
Epoch: 5, Loss: 0.33344826102256775
Epoch: 6, Loss: 0.30807045102119446
Epoch: 7, Loss: 0.2880565822124481
Epoch: 8, Loss: 0.2717464864253998
Epoch: 9, Loss: 0.25811734795570374
Epoch: 10, Loss: 0.24650143086910248
Epoch: 11, Loss: 0.23644238710403442
Epoch: 12, Loss: 0.22761695086956024
Epoch: 13, Loss: 0.21978892385959625
Epoch: 14, Loss: 0.2127813696861267
Epoch: 15, Loss: 0.2064586579799652
Epoch: 16, Loss: 0.20071512460708618
Epoch: 17, Loss: 0.19546690583229065
Epoch: 18, Loss: 0.19064649939537048
Epoch: 19, Loss: 0.1861988753080368
Epoch: 20, Loss: 0.18207862973213196
Epoch: 21, Loss: 0.17824800312519073
Epoch: 22, Loss: 0.17467515170574188
Epoch: 23, Loss: 0.17133305966854095
Epoch: 24, Loss: 0.1681986004114151
Epoch: 25, Loss: 0.16525186598300934


In [61]:
with torch.no_grad():
    y_pred = model.forward(X_test_tensor.to(torch.float32))
    y_pred = (y_pred > 0.5).float()
    accuracy = (y_pred == y_test_tensor).float().mean()
    print(f'Accuracy: {accuracy}')

Accuracy: 0.5193905830383301


In [63]:
model = MySimpleNN(X_train.shape[1])
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [64]:
## Using torch.optim optimizer for training

for epoch in range(epochs):
    y_pred = model(X_train_tensor.to(torch.float32))
    loss = loss_function(y_pred, (y_train_tensor.to(torch.float32)).view(-1, 1))
    ## Recommended that we should clear grads just before backward pass
    optimizer.zero_grad()

    loss.backward()

    optimizer.step()
    
    

    print(f'Epoch: {epoch + 1}, Loss: {loss}')

Epoch: 1, Loss: 0.588299572467804
Epoch: 2, Loss: 0.4563797414302826
Epoch: 3, Loss: 0.38568392395973206
Epoch: 4, Loss: 0.3415355682373047
Epoch: 5, Loss: 0.31078729033470154
Epoch: 6, Loss: 0.2878243625164032
Epoch: 7, Loss: 0.26983824372291565
Epoch: 8, Loss: 0.2552543580532074
Epoch: 9, Loss: 0.24311448633670807
Epoch: 10, Loss: 0.23279890418052673
Epoch: 11, Loss: 0.22388722002506256
Epoch: 12, Loss: 0.21608319878578186
Epoch: 13, Loss: 0.2091715782880783
Epoch: 14, Loss: 0.2029915153980255
Epoch: 15, Loss: 0.1974204033613205
Epoch: 16, Loss: 0.19236283004283905
Epoch: 17, Loss: 0.18774326145648956
Epoch: 18, Loss: 0.18350104987621307
Epoch: 19, Loss: 0.17958687245845795
Epoch: 20, Loss: 0.17596009373664856
Epoch: 21, Loss: 0.17258691787719727
Epoch: 22, Loss: 0.1694389432668686
Epoch: 23, Loss: 0.16649207472801208
Epoch: 24, Loss: 0.16372579336166382
Epoch: 25, Loss: 0.16112245619297028


In [65]:
with torch.no_grad():
    y_pred = model.forward(X_test_tensor.to(torch.float32))
    y_pred = (y_pred > 0.5).float()
    accuracy = (y_pred == y_test_tensor).float().mean()
    print(f'Accuracy: {accuracy}')

Accuracy: 0.5180055499076843
